In [1]:
import re
import pandas as pd
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
import os,sys
import tqdm
import pickle
import copy

In [146]:
rootPath ='D:/git/IoT_Sensors_Security_Analysis/data/perf/'
resultsPath = 'D:/git/IoT_Sensors_Security_Analysis/results/tw_50_turn_1/'

In [147]:
encoded_trace_df = pd.read_pickle(resultsPath+'encoded_bow.pkl')

Exception in thread Exception ignored in: <function _ConnectionBase.__del__ at 0x000001EB2C67E700>
Traceback (most recent call last):
  File "C:\Python39\lib\multiprocessing\connection.py", line 137, in __del__
QueueFeederThread:
Traceback (most recent call last):
  File "C:\Python39\lib\multiprocessing\queues.py", line 241, in _feed
        close()
  File "C:\Python39\lib\multiprocessing\connection.py", line 182, in close
self._close()
  File "C:\Python39\lib\multiprocessing\connection.py", line 282, in _close
    self._close()
  File "C:\Python39\lib\multiprocessing\connection.py", line 282, in _close
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid
    _CloseHandle(self._handle)
OSError: [WinError 6] The handle is invalid

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Python39\lib\threading.py", line 973, in _bootstrap_inner
    self.run()
  File "C:\Python39\lib\threading.py", line 910, i

In [148]:
pd.DataFrame([len(i) for i in encoded_trace_df['dict index encoding']]).describe()

,0
count,3932.000000
mean,126092.336216
std,11414.148432
min,99542.000000
25%,117191.250000
50%,127012.500000
75%,136056.750000
max,143245.000000


In [149]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import tqdm
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import time
import numpy as np
from sklearn import metrics
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.decomposition import PCA

In [150]:
features = ['system calls frequency' ,'system calls tfidf','system calls hashing', 'system calls dependency graph', 'one hot encoding', 'dict index encoding' ]
malwares=["delay", "disorder", "freeze", "hop", "mimic", "noise", "repeat", "spoof"]

In [151]:
normal = encoded_trace_df[encoded_trace_df.maltype=='normal']

In [152]:
def reshape_matrix(matrix_list):
    new_list = [np.array(i).reshape(-1) for i in matrix_list]
    return new_list

In [153]:
def padding_onehot(onehot_list, padding):
    new_list = []
    for onehot in onehot_list:
        if len(onehot) > padding:
            onehot = np.array(onehot[0:padding])
            new_list.append(onehot)
        else:
            onehot =np.pad(onehot, [(0, padding-len(onehot)), (0, 0)], mode='constant', constant_values=0)
            new_list.append(onehot)
    # new_list = reshape_matrix(new_list)
    return new_list

In [70]:
X = normal['one hot encoding'].tolist()
y = np.zeros(len(X))
X = padding_onehot(X, 170000)
mlb = LabelBinarizer()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, random_state=42)
X_train = [T.FloatTensor(i) for i in X_train]
X_val = [T.FloatTensor(i) for i in X_val]


In [71]:
train_loader = T.utils.data.DataLoader(X_train, batch_size=256, shuffle=True, drop_last=True, pin_memory=True, num_workers=4)
val_loader = T.utils.data.DataLoader(X_val, batch_size=256, shuffle=False, drop_last=False, num_workers=4)

In [154]:
class CNN_LSTM_Encoder(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=8):
        super(CNN_LSTM_Encoder, self).__init__()
        self.seq_len, self.n_features = seq_len, n_features
        self.embedding_dim, self.hidden_dim = embedding_dim, 4 * embedding_dim
        self.cnn = nn.Conv1d(self.seq_len, 100, 1, 1)
        self.rnn1 = nn.LSTM(
        input_size=n_features,
        hidden_size=self.hidden_dim,
        num_layers=1,
        batch_first=True
        )
        self.rnn2 = nn.LSTM(
        input_size=self.hidden_dim,
        hidden_size=embedding_dim,
        num_layers=1,
        batch_first=True
        )
    def forward(self, x):
        x = self.cnn(x)
        x, (_, _) = self.rnn1(x)
        x, (_, _) = self.rnn2(x)
        
        return x

In [155]:
class CNN_LSTM_Decoder(nn.Module):
    def __init__(self, seq_len, input_dim=8, n_features=1):
        super(CNN_LSTM_Decoder, self).__init__()
        self.seq_len, self.input_dim = seq_len, input_dim
        self.hidden_dim, self.n_features = 4 * input_dim, n_features
        self.cnn = nn.ConvTranspose1d(100,self.seq_len,1,1)
        self.rnn1 = nn.LSTM(
            input_size=input_dim,
            hidden_size=self.hidden_dim,
            num_layers=1,
            batch_first=True
        )
        self.rnn2 = nn.LSTM(
            input_size=self.hidden_dim,
            hidden_size=self.n_features,
            num_layers=1,
            batch_first=True
        )
        # self.output_layer = nn.Linear(self.hidden_dim, n_features)
        
    def forward(self, x):
        x, (_, _) = self.rnn1(x)
        x, (_, _) = self.rnn2(x)
        x = self.cnn(x)
        # x = x.reshape((self.seq_len, self.hidden_dim))
        return x

In [156]:
class Autoencoder_LSTM(nn.Module):
    def __init__(self, seq_len, n_features, embedding_dim=4):
        super(Autoencoder_LSTM, self).__init__()
        self.encoder = CNN_LSTM_Encoder(seq_len, n_features, embedding_dim)
        self.decoder = CNN_LSTM_Decoder(seq_len, embedding_dim, n_features)
        
    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [157]:
def train(train_loader, val_loader, seq_len, n_features):
    net = Autoencoder_LSTM(seq_len, n_features, 4)
    net = net.train() 
    loss_func = T.nn.MSELoss()
    optimizer = T.optim.Adam(net.parameters(), lr=0.01)
    max_epochs = 200
    print("Starting training")
    last_val_loss = 100000000
    patience = 0
    for epoch in range(0, max_epochs):
        loss = 0
        if epoch > 0 and epoch % (max_epochs/10) == 0:
            print("epoch = %6d" % epoch, end="")
            print("  prev batch loss = %7.4f, perv batch val-loss = %7.4f" %( loss/len(train_loader),val_loss/len(val_loader)))
        for curr_bat in train_loader:
            X = T.Tensor(curr_bat)
            optimizer.zero_grad()
            oupt = net(X)
            loss_obj = loss_func(oupt, X)  # note X not Y
            loss += loss_obj
            loss_obj.backward()
            optimizer.step()
        val_loss = 0
        with T.no_grad():
            for curr_bat in val_loader:
                X = T.Tensor(curr_bat)
                oupt = net(X)
                val_loss_obj = loss_func(oupt, X)  # note X not Y
                val_loss += val_loss_obj.item()
            if val_loss < last_val_loss:
                last_val_loss = val_loss
                patience = 0
            else:
                patience += 1
        if patience >= 20:
            break               
    print("Training complete")
    return net


In [158]:
def find_threshold(net, train_loader):
    net = net.eval()
    pred = []
    x_train = []
    with T.no_grad():
        for x_t in train_loader:
            y_t = net(x_t)
            x_train.append(x_t)
            pred.append(y_t)
        flat_x = np.concatenate(x_train,)
        flat_pred_outs = np.concatenate(pred,)
        y_pred = np.array([float(T.sum((T.FloatTensor(flat_x[i])-T.FloatTensor(flat_pred_outs[i]))*(T.FloatTensor(flat_x[i])-T.FloatTensor(flat_pred_outs[i])))) for i in range(0,len(flat_x))])
        down_threshold = np.percentile(y_pred, 5)
        up_threshold = np.percentile(y_pred, 95)
    return down_threshold, up_threshold

In [159]:
def test(net, test_loader, down_threshold, up_threshold):
    net = net.eval()
    pred = []
    x_test = []
    y_pred = []
    with T.no_grad():
        for x_t in test_loader:
            y_t = net(x_t)
            x_test.append(x_t)
            pred.append(y_t)
        flat_x = np.concatenate(x_test,)
        flat_pred_outs = np.concatenate(pred,)
        for i in range(0,len(flat_x)):
            t = T.sum((T.FloatTensor(flat_x[i])-T.FloatTensor(flat_pred_outs[i]))*(T.FloatTensor(flat_x[i])-T.FloatTensor(flat_pred_outs[i])))
            if t > down_threshold and t < up_threshold:
                y_pred.append(0)
            else:
                y_pred.append(1)
    return y_pred

In [161]:
resultsdict = dict()
predsdict = dict()
classifiersdict = dict()

In [162]:
# features = ['dependency_graph_features', 'one_hot_features_50', 'one_hot_features_100', 'one_hot_features_200', 'one_hot_features_500',  'one_hot_features_1000', 'one_hot_features_2000']
features = ['one hot encoding']
for feature in features:
    if 'one hot' in feature :
        X = normal['one hot encoding'].tolist()
        padding = 140000
        seq_len = padding
        X = padding_onehot(X, padding)
    else:
         X = normal[feature].tolist()
         seq_len = 15
    y = np.zeros(len(X))
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.3, random_state=42)
    X_train = [T.FloatTensor(i) for i in X_train]
    X_val = [T.FloatTensor(i) for i in X_val]
    train_loader = T.utils.data.DataLoader(X_train, batch_size=256, shuffle=False, drop_last=False, pin_memory=True, num_workers=4)
    val_loader = T.utils.data.DataLoader(X_val, batch_size=256, shuffle=False, drop_last=False, num_workers=4)

    #train stage
    n_features = 15
    t1 = time.time()
    net = train(train_loader,val_loader, seq_len, n_features)
    t2 = time.time()
    
    t = t2 -t1

    # validation stage
    down_threshold, up_threshold = find_threshold(net, train_loader)
    y_pred = test(net, val_loader, down_threshold, up_threshold)
    score = metrics.accuracy_score(y_val,y_pred)
    name = 'Autoencoder_RNN'
    res = dict()
    res['Model'] = 'valid_' + feature + '_' + name
    res['Accuracy'] = score
    res['Training time'] = t

    classifier = dict()
    classifier['Autoencoder_RNN'] = net
    classifier['down_threshold'] = down_threshold 
    classifier['up_threshold'] = up_threshold 
    print('Model: {}, accuracy score: {}, training time is: {} seconds'.format(res['Model'], score, t))

    classifiersdict[res['Model']] = classifier
    resultsdict[res['Model']] = res
    predsdict[res['Model']] = y_pred
    
    # testing stage
    for malware in malwares:
        dfs = encoded_trace_df[encoded_trace_df.maltype==malware]
        if 'one hot' in feature :
            X = dfs['one hot encoding'].tolist()
            padding = 140000
            seq_len = padding
            X = padding_onehot(X, padding)
        else:
            X = dfs[feature].tolist()
            seq_len = 15
        mlb = LabelBinarizer()
        y_test = np.ones(len(X))
        X_test = [T.FloatTensor(i) for i in X]
        
        test_loader =  T.utils.data.DataLoader(X_test, batch_size=256, shuffle=False, drop_last=False, num_workers=4)
        t1 =time.time()
        y_pred = test(net, test_loader, down_threshold, up_threshold)
        t2 =time.time()
        score = metrics.accuracy_score(y_test,y_pred)
        t = t2 -t1
        pred = dict()    
        name = 'Autoencoder_RNN'
        res = dict()

        pred[malware +'_' + feature + '_' + name] = y_pred

        res['Model'] = malware +'_' + feature + '_' + name
        res['Accuracy'] = score
        res['Testing time'] = t


        resultsdict[res['Model']] = res
        predsdict[res['Model']] = y_pred   
        print('Model: {}, accuracy score: {}, testing time is: {} seconds'.format( res['Model'], score, t))
        
loc=open(resultsPath+'rnn_classifiers.pk','wb')
pickle.dump(classifiersdict,loc)
loc=open(resultsPath+'rnn_results.pk','wb')
pickle.dump(resultsdict,loc)
loc=open(resultsPath+'rnn_preds.pk','wb')
pickle.dump(predsdict,loc)   


Starting training
epoch =     20  prev batch loss =  0.0000, perv batch val-loss =  0.0384
epoch =     40  prev batch loss =  0.0000, perv batch val-loss =  0.0384
Training complete
Model: valid_one hot encoding_Autoencoder_RNN, accuracy score: 0.8991596638655462, training time is: 1287.4521706104279 seconds
Model: delay_one hot encoding_Autoencoder_RNN, accuracy score: 0.22010869565217392, testing time is: 20.770519495010376 seconds
Model: disorder_one hot encoding_Autoencoder_RNN, accuracy score: 0.1590909090909091, testing time is: 21.942049741744995 seconds
Model: freeze_one hot encoding_Autoencoder_RNN, accuracy score: 0.1893939393939394, testing time is: 22.01976203918457 seconds
Model: hop_one hot encoding_Autoencoder_RNN, accuracy score: 0.13131313131313133, testing time is: 22.31789207458496 seconds
Model: mimic_one hot encoding_Autoencoder_RNN, accuracy score: 0.18686868686868688, testing time is: 21.838648080825806 seconds
Model: noise_one hot encoding_Autoencoder_RNN, accur

In [163]:
rd = []
for m in resultsdict:
    rd.append(resultsdict[m])

In [164]:
rrd = pd.DataFrame(rd, index=None)

In [165]:
md = [i.split('_') for i in rrd['Model']]
md = pd.DataFrame(md)

In [166]:
md.columns  = ['Dataset','Features','Model','Architecture']

In [167]:
nrd=pd.DataFrame([md['Dataset'],md['Features'],md['Model'],md['Architecture'], rrd['Accuracy']]).transpose()

In [168]:
nrd.to_csv(resultsPath+'Autoencoder_RNN_results.csv',index=None)

In [178]:
a = ['1d 2sd 3f 4f', '1d 4f 3f']

In [188]:
ft = TfidfVectorizer().fit_transform(a)

In [189]:
ft.data

array([0.44832087, 0.44832087, 0.63009934, 0.44832087, 0.57735027,
       0.57735027, 0.57735027])

In [187]:
ft[0][0]

<1x4 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>